In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from scipy.stats import entropy

sns.set_style()
%matplotlib inline

In [ ]:
fid_path = '/path/to/FID/'
path = '/path/to/logs/'

## FID loggigs

In [ ]:
fid = []
itrs_fid = []

with open(fid_path + 'BGd_163_log.jsonl') as f:
    for line in f.readlines():
        print(line)

## Loss for G & D

In [ ]:
G_loss = []
D_loss_real = []
D_loss_fake = []
steps = []

with open(path + 'G_loss.log') as f:
    for line in f.readlines():
        G_loss.append(float(line.split(':')[1]))
        steps.append(int(line.split(':')[0]))

with open(path + 'D_loss_real.log') as f:
    for line in f.readlines():
        D_loss_real.append(float(line.split(':')[1]))


with open(path + 'D_loss_fake.log') as f:
    for line in f.readlines():
        D_loss_fake.append(float(line.split(':')[1]))

avg = np.convolve(G_loss, np.ones(16)/16, mode='valid')
avdf = np.convolve(D_loss_fake, np.ones(16)/16, mode='valid')
avdr = np.convolve(D_loss_real, np.ones(16)/16, mode='valid')
avs = np.convolve(steps, np.ones(16)/16, mode='valid')
# Keep one every 8 elements (TPU cores)
G_loss = G_loss[::8]
D_loss_real = D_loss_real[::8]
D_loss_fake = D_loss_fake[::8]
steps = steps[::8]

plt.figure(figsize=(15, 8))
sns.set_style("darkgrid")
plt.plot(steps, G_loss)
plt.plot(steps, D_loss_real)
plt.plot(steps, D_loss_fake)
plt.plot(avs, avg)
plt.plot(avs, avdf)
# plt.plot(avs,avdr)
plt.legend(['G_loss', 'D_loss_real', 'D_loss_fake',
           'mvg_G', 'mvg_D_fake'], loc='upper right')
plt.ylim((0, 8))
plt.show()

In [ ]:
en_40k= entropy(avg[:40000],avdf[:40000])
en_curr = entropy(avg,avdf)
print(f'The current entropy is:{en_curr}, while the entropy at 40k is:{en_40k}')

## Spectral Analysis for D

In [ ]:
# Singular Value Analysis for D
files = {}
i = 0
for file in os.listdir(path):
    if file.startswith('D'):
        lines = []
        itrs = []
        with open(file) as f:
            for line in f:
                fields = line.replace(":", "").split()
                lines.append(float(fields[1]))
                itrs.append(int(fields[0]))
            files[file] = lines

In [ ]:
l_f = files.pop('D_loss_fake.log')
l_r = files.pop('D_loss_real.log')
lists = sorted(files.items())
x, y = zip(*lists)
df= pd.DataFrame.from_dict(files)
df['itr']=np.arange(10,(len(lines)*10)+1,10)
df_melted = df.melt("itr", var_name="SVs", value_name="value")

In [ ]:
sns.set_style("darkgrid")
fig = px.line(df_melted, x="itr", y="value", color='SVs')
fig.show()

## Spectral Analysis for G

In [ ]:
# Spectral Analysis for G
files_G = {}
i = 0
for file in os.listdir(path):
    if file.startswith('G'):
        lines_G = []
        itrs_G = []
        with open(file) as f:
            for line in f:
                fields_G = line.replace(":", "").split()
                lines_G.append(float(fields_G[1]))
                itrs_G.append(int(fields_G[0]))
            files_G[file] = lines_G

In [ ]:
l_g = files_G.pop('G_loss.log')
dg = pd.DataFrame.from_dict(files_G)
dg['itr'] = np.arange(10,(dg.shape[0]*10)+1,10)
dg_melted = dg.melt("itr", var_name="SVs", value_name="value")

In [ ]:
fig = px.line(dg_melted, x="itr", y="value", color='SVs', log_x=False, log_y=False)
fig.show()

## Loss for D and G individually

In [ ]:
G_loss = []
D_loss_real = []
D_loss_fake = []
steps = []

with open(path + 'G_loss.log') as f:
    for line in f.readlines():
        G_loss.append(float(line.split(':')[1]))
        steps.append(int(line.split(':')[0]))

with open(path + 'D_loss_real.log') as f:
    for line in f.readlines():
        D_loss_real.append(float(line.split(':')[1]))


with open(path + 'D_loss_fake.log') as f:
    for line in f.readlines():
        D_loss_fake.append(float(line.split(':')[1]))
        
        
G_loss = G_loss[::8]
D_loss_real = D_loss_real[::8]
D_loss_fake = D_loss_fake[::8]
steps = steps[::8]

In [ ]:
fig = px.line(x=steps, y=G_loss, log_x=False, log_y=False)
fig['data'][0]['line']['color']="#cc0000"
fig.show()

In [ ]:
fig = px.line(x=steps, y=D_loss_real, log_x=False, log_y=False)
fig.show()

In [ ]:
fig = px.line(x=steps, y=D_loss_fake, log_x=False, log_y=False)
fig.show()

# Uniform & IEA Loss Analysis

In [ ]:
unif_loss = []
iea_loss = []
steps = []

with open(path + 'unif_loss_d.log') as f:
    for line in f.readlines():
        unif_loss.append(float(line.split(':')[1]))
        steps.append(int(line.split(':')[0]))

with open(path + 'iep_loss.log') as f:
    for line in f.readlines():
        iep_loss.append(float(line.split(':')[1]))

        

unif_loss = unif_loss[::8]
icp_loss = iep_loss[::8]
steps = steps[::8]

In [ ]:
fig = px.line(x=steps, y=unif_loss, log_x=False, log_y=False)
fig['data'][0]['line']['color']="#cc0000"
fig.show()

In [ ]:
fig = px.line(x=steps, y=iea_loss, log_x=False, log_y=False)
fig.show()